# Make the edge between nodes

In [13]:
import csv
import pandas as pd

#### Load node file

In [14]:
node = pd.read_csv('/users/noah/desktop/nodes/node1.csv')
#node = pd.read_csv('/users/noah/desktop/nodes/totalnode.csv')

#### Load text and score to dataframe

In [15]:
df1 = pd.read_csv('/users/noah/desktop/hp1_score.csv')
df2 = pd.read_csv('/users/noah/desktop/hp2_score.csv')
df3 = pd.read_csv('/users/noah/desktop/hp3_score.csv')
df4 = pd.read_csv('/users/noah/desktop/hp4_score.csv')
df5 = pd.read_csv('/users/noah/desktop/hp5_score.csv')
df6 = pd.read_csv('/users/noah/desktop/hp6_score.csv')
df7 = pd.read_csv('/users/noah/desktop/hp7_score.csv')
df8 = pd.read_csv('/users/noah/desktop/hp8_score.csv')

#### Concat the dataframes and make it to one dataframe

In [18]:
scores = df1
#scores = pd.concat((df1,df2,df3,df4,df5,df6,df7,df8), sort=False, ignore_index = True)

#### Replace the NULL or NA

In [19]:
scores['listener'].fillna('', inplace = True)

In [20]:
from itertools import permutations
import numpy as np
import time

#### Make source and target by permutation function. Make weight, count and mean variable to use later

In [21]:
edge_base = pd.DataFrame(list(permutations(node['Label'], 2)))
edge_base.rename(columns = {0:'source', 1:'target'}, inplace = True)
weight = np.zeros(len(edge_base)) 
count = np.zeros(len(edge_base))
mean = np.zeros(len(edge_base))
print(edge_base)

                   source               target
0            Harry Potter          Ron Weasley
1            Harry Potter     Hermione Granger
2            Harry Potter     Albus Dumbledore
3            Harry Potter        Rubeus Hagrid
4            Harry Potter        Severus Snape
...                   ...                  ...
41407  Minerva Mcgonagall  Gellert Grindelward
41408  Minerva Mcgonagall          Ron Weasley
41409  Minerva Mcgonagall              Bogrod 
41410  Minerva Mcgonagall    Ariana Dumbledore
41411  Minerva Mcgonagall  Neville Longbottom 

[41412 rows x 2 columns]


#### Make copy of edge_base and turn into the list

In [22]:
edge_test = edge_base.copy()
edge_list = edge_test.values.tolist()

### Find the weight and count. Then calculate the mean sentimental value between source and target

In [23]:
start_time = time.time()

for i in range(len(edge_list)):
    
    print(f"{i+1}/{len(edge_list)}", end='\r')
                   
    for j in range(len(scores)):
        
        if ((edge_list[i][1] in scores['listener'][j]) & (edge_list[i][0] == scores['character'][j])):
            
            count[i] +=1                                        
            weight[i] += scores['score'][j]
            
        else:
            weight[i] += 0
    
        if count[i] != 0:
            mean[i] = weight[i] / count[i]
    
end_time = time.time()

print(f"Time is {end_time - start_time} sec")                       

Time is 13753.822365045547 sec


#### Make them to the dataframe

In [24]:
edge_base['mean'] = mean
edge_base['weight'] = weight
edge_base['count'] = count
print(edge_base)

                   source               target      mean  weight  count
0            Harry Potter          Ron Weasley -0.011111  -4.589  413.0
1            Harry Potter     Hermione Granger -0.022344  -9.429  422.0
2            Harry Potter     Albus Dumbledore -0.120302 -17.925  149.0
3            Harry Potter        Rubeus Hagrid -0.161384 -13.879   86.0
4            Harry Potter        Severus Snape -0.083850  -3.354   40.0
...                   ...                  ...       ...     ...    ...
41407  Minerva Mcgonagall  Gellert Grindelward  0.000000   0.000    0.0
41408  Minerva Mcgonagall          Ron Weasley  0.000000   0.000    0.0
41409  Minerva Mcgonagall              Bogrod   0.000000   0.000    0.0
41410  Minerva Mcgonagall    Ariana Dumbledore  0.000000   0.000    0.0
41411  Minerva Mcgonagall  Neville Longbottom   0.000000   0.000    0.0

[41412 rows x 5 columns]


#### Use some techniques to what we want to make finanlly

In [25]:
edge_id = pd.DataFrame(list(permutations(node['Id'], 2)))
edge_id.rename(columns = {0:'source', 1:'target'}, inplace = True)
edge_id['mean'] = mean
print(edge_id)

       source  target      mean
0           0       1 -0.011111
1           0       2 -0.022344
2           0       3 -0.120302
3           0       4 -0.161384
4           0       5 -0.083850
...       ...     ...       ...
41407     203     198  0.000000
41408     203     199  0.000000
41409     203     200  0.000000
41410     203     201  0.000000
41411     203     202  0.000000

[41412 rows x 3 columns]


#### Save it to edge csv file

In [26]:
edge_id.to_csv('/users/noah/desktop/edge/totaledge.csv', index = False)

### Repeat 1 to 8 and total to make all edge files